# Notebook 02: Region Proposals & Selective Search

**Week 15 - Module 5: Object Detection**

**Duration:** ~15 minutes

**Learning Objectives:**
- Understand the region proposal concept and why it matters
- Learn how Selective Search algorithm works
- Implement Selective Search hands-on
- Understand why Region Proposal Network (RPN) replaced it

## 1. What are Region Proposals?

### The Object Detection Challenge:

Given an image, we need to find all objects. But...

**Naive Approach - Sliding Window:**
- Try every possible bounding box position
- Multiple scales: 50×50, 100×100, 200×200, ...
- Multiple aspect ratios: 1:1, 1:2, 2:1, ...
- **Result**: Millions of candidate boxes per image!

**Example Calculation:**
- Image: 640×480 pixels
- Scales: 10 different sizes
- Aspect ratios: 3 ratios
- Stride: 16 pixels
- **Total boxes**: (640/16) × (480/16) × 10 × 3 = **36,000 boxes**

Running a CNN classifier 36,000 times per image is impractical!

### Region Proposals Solution:

**Key Idea**: Pre-filter candidate regions before expensive classification

**Goals:**
1. **Reduce search space**: Millions → Thousands
2. **High recall**: Don't miss actual objects (recall ≥ 95%)
3. **Class-agnostic**: Find "object-like" regions regardless of category
4. **Fast generation**: < 1 second per image

**Analogy**: 
- Region proposals = "Where might objects be?"
- Like a metal detector scanning beach before digging
- Focus expensive computation where it matters

### Historical Methods:

1. **Objectness (2010)**: Generic object saliency
2. **Selective Search (2011-2013)**: Hierarchical grouping (used in R-CNN)
3. **EdgeBoxes (2014)**: Contour-based proposals
4. **RPN (2015)**: Learned neural network proposals (Faster R-CNN) ← Current standard

## 2. Selective Search Algorithm

**Paper**: "Selective Search for Object Recognition" (2013)

**Authors**: J.R.R. Uijlings et al.

### Core Philosophy:

Objects can appear at different scales and have various characteristics. Use **multiple strategies** and combine them:

1. **Hierarchical Grouping**: Start small, merge similar regions
2. **Multiple Similarity Measures**: Color, texture, size, fill
3. **Multiple Color Spaces**: RGB, HSV, Lab, etc.
4. **Diversification**: Capture all possible object-like regions

### Algorithm Steps:

```
Step 1: Initial Segmentation
  ├─ Use graph-based segmentation (Felzenszwalb)
  ├─ Over-segment image into ~1000 small regions
  └─ Each region = potential object part

Step 2: Calculate Similarity
  ├─ Color similarity (histogram intersection)
  ├─ Texture similarity (Gaussian derivatives)
  ├─ Size similarity (prefer merging small regions)
  └─ Fill similarity (how well regions fit together)

Step 3: Hierarchical Grouping
  ├─ Merge most similar neighboring regions
  ├─ Recalculate similarities
  ├─ Repeat until entire image is one region
  └─ Record all merge steps

Step 4: Generate Proposals
  ├─ Each region in hierarchy = one proposal
  ├─ Compute bounding box for each region
  └─ Output: ~2000 region proposals
```

### Similarity Metrics:

**1. Color Similarity** (s_color):
- 25-bin histogram per color channel
- Histogram intersection: s_color(ri, rj) = Σ min(ci, cj)

**2. Texture Similarity** (s_texture):
- SIFT-like features: Gaussian derivatives in 8 orientations
- 10-bin histogram per channel per orientation

**3. Size Similarity** (s_size):
- s_size(ri, rj) = 1 - (size(ri) + size(rj)) / size(image)
- Encourages merging small regions first

**4. Fill Similarity** (s_fill):
- s_fill(ri, rj) = 1 - (size(BB_merge) - size(ri) - size(rj)) / size(image)
- BB_merge = bounding box of merged region
- Prefers regions that fit together well

**Combined Similarity**:
```
s(ri, rj) = a1*s_color + a2*s_texture + a3*s_size + a4*s_fill
```

### Fast vs Quality Modes:

**Fast Mode** (default):
- Fewer color spaces (just RGB)
- Fewer starting regions
- ~1000 proposals in 1 second

**Quality Mode**:
- Multiple color spaces (RGB, HSV, Lab, etc.)
- More diversified strategies
- ~2000 proposals in 2-3 seconds
- Better recall (fewer missed objects)

In [ ]:
# Setup
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import time

print("OpenCV version:", cv2.__version__)
print("\nNote: Selective Search requires opencv-contrib-python")
print("Install: pip install opencv-contrib-python")

In [ ]:
# Helper function to create sample image
def create_sample_image():
    """Create a simple test image with colored rectangles"""
    img = np.ones((400, 600, 3), dtype=np.uint8) * 255  # White background
    
    # Draw colored rectangles (simulating objects)
    cv2.rectangle(img, (50, 50), (150, 150), (255, 0, 0), -1)     # Blue square
    cv2.rectangle(img, (200, 100), (400, 200), (0, 255, 0), -1)   # Green rectangle
    cv2.rectangle(img, (450, 250), (550, 350), (0, 0, 255), -1)   # Red square
    cv2.circle(img, (300, 300), 50, (255, 255, 0), -1)            # Yellow circle
    
    return img

# Create and display sample image
img = create_sample_image()
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(10, 6))
plt.imshow(img_rgb)
plt.title('Sample Image with 4 Objects', fontsize=14, fontweight='bold')
plt.axis('off')
plt.tight_layout()
plt.show()

print(f"Image shape: {img.shape}")
print(f"Number of actual objects: 4")

In [ ]:
# Implement Selective Search

def run_selective_search(image, mode='fast'):
    """
    Run Selective Search on image
    
    Args:
        image: Input image (BGR format)
        mode: 'fast' or 'quality'
    
    Returns:
        proposals: Array of bounding boxes [x, y, w, h]
        elapsed_time: Processing time in seconds
    """
    # Create Selective Search object
    ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
    ss.setBaseImage(image)
    
    # Choose mode
    if mode == 'fast':
        ss.switchToSelectiveSearchFast()
    else:
        ss.switchToSelectiveSearchQuality()
    
    # Run Selective Search
    start_time = time.time()
    proposals = ss.process()
    elapsed_time = time.time() - start_time
    
    return proposals, elapsed_time

# Run on sample image
print("Running Selective Search (Fast mode)...")
proposals_fast, time_fast = run_selective_search(img, mode='fast')

print(f"\nResults:")
print(f"  Number of proposals: {len(proposals_fast)}")
print(f"  Processing time: {time_fast:.3f} seconds")
print(f"  Proposals per second: {len(proposals_fast)/time_fast:.1f}")

# Show first few proposals
print(f"\nFirst 5 proposals (x, y, w, h):")
for i, (x, y, w, h) in enumerate(proposals_fast[:5]):
    print(f"  {i+1}. [{x:4d}, {y:4d}, {w:4d}, {h:4d}] - Area: {w*h:6d} pixels")

In [ ]:
# Visualize Region Proposals

def visualize_proposals(image, proposals, max_proposals=100, title="Region Proposals"):
    """
    Visualize region proposals on image
    """
    img_rgb = cv2.cvtColor(image.copy(), cv2.COLOR_BGR2RGB)
    
    fig, ax = plt.subplots(figsize=(12, 8))
    ax.imshow(img_rgb)
    
    # Draw proposals
    for i, (x, y, w, h) in enumerate(proposals[:max_proposals]):
        rect = Rectangle((x, y), w, h, linewidth=1, 
                        edgecolor='lime', facecolor='none', alpha=0.4)
        ax.add_patch(rect)
    
    ax.set_title(f'{title}\n(Showing {min(max_proposals, len(proposals))}/{len(proposals)} proposals)',
                fontsize=14, fontweight='bold')
    ax.axis('off')
    plt.tight_layout()
    plt.show()

# Visualize top 100 proposals
visualize_proposals(img, proposals_fast, max_proposals=100, 
                   title="Selective Search - Top 100 Proposals")

print("Observation: Notice how proposals cover objects at different scales")
print("Green boxes = potential object regions identified by algorithm")

In [ ]:
# Analyze Proposals - Size Distribution

# Calculate proposal statistics
areas = [w * h for (x, y, w, h) in proposals_fast]
aspect_ratios = [w / h if h > 0 else 0 for (x, y, w, h) in proposals_fast]
widths = [w for (x, y, w, h) in proposals_fast]
heights = [h for (x, y, w, h) in proposals_fast]

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Area distribution
axes[0, 0].hist(areas, bins=50, color='skyblue', edgecolor='black', alpha=0.7)
axes[0, 0].set_xlabel('Area (pixels²)', fontsize=11, fontweight='bold')
axes[0, 0].set_ylabel('Frequency', fontsize=11, fontweight='bold')
axes[0, 0].set_title('Proposal Area Distribution', fontsize=12, fontweight='bold')
axes[0, 0].axvline(np.median(areas), color='red', linestyle='--', 
                   label=f'Median: {np.median(areas):.0f}')
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

# Aspect ratio distribution
axes[0, 1].hist(aspect_ratios, bins=50, color='lightcoral', edgecolor='black', alpha=0.7)
axes[0, 1].set_xlabel('Aspect Ratio (w/h)', fontsize=11, fontweight='bold')
axes[0, 1].set_ylabel('Frequency', fontsize=11, fontweight='bold')
axes[0, 1].set_title('Aspect Ratio Distribution', fontsize=12, fontweight='bold')
axes[0, 1].axvline(1.0, color='green', linestyle='--', label='Square (1:1)')
axes[0, 1].legend()
axes[0, 1].grid(alpha=0.3)
axes[0, 1].set_xlim([0, 5])  # Limit x-axis for better visualization

# Width vs Height scatter
axes[1, 0].scatter(widths, heights, alpha=0.3, s=10, color='purple')
axes[1, 0].set_xlabel('Width (pixels)', fontsize=11, fontweight='bold')
axes[1, 0].set_ylabel('Height (pixels)', fontsize=11, fontweight='bold')
axes[1, 0].set_title('Width vs Height', fontsize=12, fontweight='bold')
axes[1, 0].plot([0, max(widths)], [0, max(heights)], 'r--', alpha=0.5, label='Square line')
axes[1, 0].legend()
axes[1, 0].grid(alpha=0.3)

# Cumulative area coverage
sorted_areas = sorted(areas, reverse=True)
cumulative = np.cumsum(sorted_areas) / sum(areas) * 100
axes[1, 1].plot(cumulative, color='darkgreen', linewidth=2)
axes[1, 1].set_xlabel('Number of Proposals', fontsize=11, fontweight='bold')
axes[1, 1].set_ylabel('Cumulative Area Coverage (%)', fontsize=11, fontweight='bold')
axes[1, 1].set_title('Cumulative Area Coverage', fontsize=12, fontweight='bold')
axes[1, 1].axhline(80, color='red', linestyle='--', alpha=0.5, label='80% coverage')
axes[1, 1].legend()
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('proposal_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nProposal Statistics:")
print(f"  Total proposals: {len(proposals_fast)}")
print(f"  Area range: {min(areas)} - {max(areas)} pixels²")
print(f"  Median area: {np.median(areas):.0f} pixels²")
print(f"  Aspect ratio range: {min(aspect_ratios):.2f} - {max(aspect_ratios):.2f}")
print(f"  Median aspect ratio: {np.median(aspect_ratios):.2f}")

In [ ]:
# Quality vs Speed Mode Comparison

print("Comparing Fast vs Quality modes...\n")

# Run both modes
proposals_fast, time_fast = run_selective_search(img, mode='fast')
proposals_quality, time_quality = run_selective_search(img, mode='quality')

# Comparison table
comparison_data = {
    'Mode': ['Fast', 'Quality'],
    'Proposals': [len(proposals_fast), len(proposals_quality)],
    'Time (s)': [f'{time_fast:.3f}', f'{time_quality:.3f}'],
    'FPS': [f'{1/time_fast:.2f}', f'{1/time_quality:.2f}']
}

print("Mode Comparison:")
print("="*60)
print(f"{'Mode':<12} {'Proposals':<12} {'Time (s)':<12} {'FPS':<12}")
print("="*60)
for i in range(len(comparison_data['Mode'])):
    print(f"{comparison_data['Mode'][i]:<12} "
          f"{comparison_data['Proposals'][i]:<12} "
          f"{comparison_data['Time (s)'][i]:<12} "
          f"{comparison_data['FPS'][i]:<12}")
print("="*60)

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Fast mode
img_fast = cv2.cvtColor(img.copy(), cv2.COLOR_BGR2RGB)
axes[0].imshow(img_fast)
for i, (x, y, w, h) in enumerate(proposals_fast[:50]):
    rect = Rectangle((x, y), w, h, linewidth=1, 
                    edgecolor='lime', facecolor='none', alpha=0.4)
    axes[0].add_patch(rect)
axes[0].set_title(f'Fast Mode\n{len(proposals_fast)} proposals in {time_fast:.3f}s', 
                 fontsize=12, fontweight='bold')
axes[0].axis('off')

# Quality mode
img_quality = cv2.cvtColor(img.copy(), cv2.COLOR_BGR2RGB)
axes[1].imshow(img_quality)
for i, (x, y, w, h) in enumerate(proposals_quality[:50]):
    rect = Rectangle((x, y), w, h, linewidth=1, 
                    edgecolor='cyan', facecolor='none', alpha=0.4)
    axes[1].add_patch(rect)
axes[1].set_title(f'Quality Mode\n{len(proposals_quality)} proposals in {time_quality:.3f}s', 
                 fontsize=12, fontweight='bold')
axes[1].axis('off')

plt.tight_layout()
plt.savefig('fast_vs_quality.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nKey Differences:")
print(f"  Quality mode generates {len(proposals_quality) - len(proposals_fast)} more proposals")
print(f"  Quality mode is {time_quality/time_fast:.1f}× slower")
print(f"  Trade-off: Better recall vs processing time")

## 8. Problems with Selective Search

Despite being the foundation of R-CNN and Fast R-CNN, Selective Search has fundamental limitations:

### Problem 1: Not Learned from Data
- **Fixed algorithm**: Hand-crafted rules, can't adapt to dataset
- **No optimization**: Parameters manually tuned, not learned
- **Generic**: Doesn't leverage domain knowledge
- **Example**: Same strategy for faces, cars, text - but these need different approaches!

### Problem 2: Slow Processing
- **CPU-only**: No GPU acceleration
- **1-2 seconds per image**: 87% of total detection time in Fast R-CNN!
- **Bottleneck**: Can't leverage parallel processing
- **Compare**: Modern RPN runs in milliseconds on GPU

### Problem 3: Low-Quality Proposals
- **Many redundant boxes**: High overlap between proposals
- **Poor localization**: Bounding boxes not tight around objects
- **Recall-precision trade-off**: Need 2000 proposals to get 95% recall
- **Missed objects**: Small or unusual objects often missed

### Problem 4: Not End-to-End Trainable
- **Separate preprocessing step**: Can't train jointly with detector
- **No gradient flow**: Can't optimize for detection task
- **Fixed during training**: Detector adapts around proposal limitations

### Problem 5: No Task-Specific Optimization
- **Generic "objectness"**: Not optimized for specific classes
- **Fixed hierarchy**: Can't learn which merges matter
- **No feedback**: Detection errors don't improve proposals

### Quantitative Limitations:

| Metric | Selective Search | RPN (Faster R-CNN) |
|--------|------------------|--------------------|
| **Processing Time** | 1-2 seconds | 10 milliseconds |
| **Proposals Needed** | ~2000 | ~300 |
| **Recall @ 95%** | 2000 proposals | 300 proposals |
| **Trainable** | No | Yes |
| **GPU Accelerated** | No | Yes |
| **Localization Quality** | Moderate | High |
| **Adaptable** | No | Yes (learns from data) |

### Real-World Impact:

**Fast R-CNN timing breakdown:**
- Selective Search: **2.0 seconds** (87%)
- CNN forward pass: 0.2 seconds (9%)
- ROI pooling + detection: 0.1 seconds (4%)
- **Total: 2.3 seconds per image**

**Faster R-CNN timing breakdown:**
- RPN: **0.01 seconds** (5%)
- CNN forward pass: 0.15 seconds (75%)
- ROI pooling + detection: 0.04 seconds (20%)
- **Total: 0.2 seconds per image**

**Result**: 11× speedup just by replacing Selective Search with RPN!

## 9. Solution: Region Proposal Network (RPN)

Faster R-CNN introduced RPN to solve all Selective Search problems:

### RPN Key Ideas:

#### 1. Learned from Data
```python
# RPN is a neural network trained end-to-end
# Input: Feature map from backbone CNN
# Output: Objectness scores + bounding box coordinates

class RPN(nn.Module):
    def __init__(self):
        self.conv = nn.Conv2d(512, 512, 3, padding=1)  # 3×3 conv
        self.cls_layer = nn.Conv2d(512, 2*k, 1)  # k anchors × 2 (obj/not)
        self.reg_layer = nn.Conv2d(512, 4*k, 1)  # k anchors × 4 (x,y,w,h)
    
    def forward(self, features):
        x = F.relu(self.conv(features))
        objectness = self.cls_layer(x)  # Is there an object?
        bbox_deltas = self.reg_layer(x)  # Where is it?
        return objectness, bbox_deltas
```

#### 2. Anchor Boxes Concept
- Pre-defined reference boxes at each feature map location
- Multiple scales: {128², 256², 512²}
- Multiple aspect ratios: {1:1, 1:2, 2:1}
- Total: 3 × 3 = 9 anchors per position

#### 3. Shared Computation
- Uses same CNN features as detector
- No separate image processing
- GPU-accelerated, parallel processing

#### 4. End-to-End Training
- Trained jointly with detection network
- Gradient flows from detection loss to RPN
- Learns what proposals help detection

### RPN vs Selective Search:

**Speed:**
- Selective Search: 2000ms
- RPN: 10ms
- **Speedup: 200×**

**Quality:**
- Selective Search: 2000 proposals for 95% recall
- RPN: 300 proposals for 96% recall
- **Efficiency: 6.7× fewer proposals, better recall**

**Adaptability:**
- Selective Search: Fixed for all datasets
- RPN: Learns optimal proposals for each dataset

### Why RPN Won:

1. **Speed**: 200× faster
2. **Quality**: Better proposals with fewer candidates
3. **Integration**: Seamless with detection pipeline
4. **Flexibility**: Adapts to any dataset/task
5. **Simplicity**: One network vs complex preprocessing

### Legacy:

RPN's anchor box concept influenced:
- **YOLO** (grid-based anchors)
- **SSD** (multi-scale anchors)
- **RetinaNet** (focal loss with anchors)
- **Modern detectors**: Most use learned proposal mechanisms

**Preview**: In Notebook 03, we'll implement Faster R-CNN with RPN!

## 10. Exercise: Generate Proposals for Your Image

**Task**: Apply Selective Search to your own image and analyze results.

### Instructions:

1. Load your image (or use provided test image)
2. Run Selective Search (both Fast and Quality modes)
3. Visualize top 100 proposals
4. Analyze:
   - How many proposals contain actual objects?
   - How many are redundant?
   - Quality of bounding boxes (tight fit?)

### Questions:

1. **What percentage of proposals are useful?**
   - Count proposals with objects vs total
   
2. **How does image complexity affect proposal count?**
   - Simple scene vs cluttered scene
   
3. **Is Quality mode worth the extra time?**
   - Compare recall improvement vs speed cost
   
4. **How would you filter proposals before detection?**
   - Size thresholds?
   - Non-maximum suppression?

### Starter Code:

```python
# Load your image
img = cv2.imread('your_image.jpg')

# Run Selective Search
proposals, time_taken = run_selective_search(img, mode='fast')

# Visualize
visualize_proposals(img, proposals, max_proposals=100)

# Analyze
print(f"Total proposals: {len(proposals)}")
print(f"Processing time: {time_taken:.3f}s")

# Your analysis here...
```

## 11. Summary

### What We Learned:

1. **Region Proposals Concept**:
   - Pre-filter candidate regions before expensive classification
   - Reduce search space from millions to thousands
   - Goal: High recall with manageable number of proposals

2. **Selective Search Algorithm**:
   - Hierarchical grouping based on color, texture, size, fill
   - Generates ~2000 diverse proposals
   - Fast vs Quality modes (speed vs recall trade-off)

3. **Selective Search Limitations**:
   - Not learned (fixed algorithm)
   - Slow (1-2 seconds per image)
   - Low-quality proposals (many redundant)
   - Not end-to-end trainable

4. **RPN Solution**:
   - Learned neural network for proposals
   - 200× faster (10ms vs 2000ms)
   - Better quality with fewer proposals
   - End-to-end trainable

### Historical Context:

- **2011-2013**: Selective Search state-of-the-art
- **2014**: R-CNN uses Selective Search → 53% mAP
- **2015**: Fast R-CNN still uses Selective Search → bottleneck identified
- **2015**: Faster R-CNN replaces with RPN → 200× speedup!
- **2015-present**: Learned proposals become standard

### Key Takeaway:

**Selective Search** was crucial for early object detection success, but **learned proposals (RPN)** proved far superior. Understanding both helps appreciate why modern detectors are designed the way they are.

### Next Notebook Preview:

**Notebook 03**: Faster R-CNN Pre-trained Implementation
- Load pre-trained Faster R-CNN
- Run detection on images
- Analyze RPN outputs
- Compare with YOLO performance

---

**Estimated completion time**: 15 minutes

**Key insight**: The evolution from Selective Search to RPN shows how deep learning replaced hand-crafted algorithms in computer vision. This pattern repeated across many CV tasks (features, detection, segmentation, etc.).